# **Init**

In [1]:
import PIL
import time
import torch
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

from ast import literal_eval
from tqdm.notebook import tqdm
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from matplotlib.image import imread

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [3]:
ROOT = Path("/content/drive/My Drive/Competition/SKELY")

!ls "/content/drive/My Drive/Competition/SKELY" 

data  models  notebooks


In [4]:
torch.cuda.device(0)
warnings.filterwarnings("ignore")
torch.multiprocessing.freeze_support()
print("[INFO] GPU:", torch.cuda.get_device_name())

[INFO] GPU: Tesla T4


In [5]:
torch.cuda.empty_cache()

## **Data Loading**

In [6]:
train_df = pd.read_csv(ROOT/'data/train/train.csv')
test_df = pd.read_csv(ROOT/'data/test/test.csv')

print("[INFO] TRAIN CLASSES: ", train_df.columns.to_list())
print("[INFO] TESR CLASSES: ", test_df.columns.to_list())

display(train_df.head(3))
display(test_df.head(3))

[INFO] TRAIN CLASSES:  ['filename', 'xRot']
[INFO] TESR CLASSES:  ['filename']


,filename,xRot
0,000000.png,182.835227
1,000001.png,246.988824
2,000002.png,280.845321


,filename
0,009999.png
1,010000.png
2,010001.png


In [7]:
def after_open(img):
    img = img.crop((500, 250, 1500, 1898))
    img.thumbnail((1024,1024))
    return img

data_list = (ImageList.from_df(path=ROOT, df=train_df, cols=['filename'], folder='data/train/train_images', convert_mode='L', after_open=after_open)
                            .split_by_rand_pct(0.1)
                            .label_from_df(cols=['xRot'], label_cls=FloatList))
test_list = (ImageList.from_df(path=ROOT, df=test_df, cols=['filename'], folder='data/test/test_images', convert_mode='L', after_open=after_open))
data_list.add_test(test_list)

augs = [RandTransform(tfm=TfmLighting (contrast), kwargs={'scale': 1.1}, p=1, resolved={}, do_run=True, is_random=False, use_on_y=False)]
tfms = (augs, augs)

def get_data(image_size, batch_size):
    size = (image_size, round(image_size / 1.651))
    data = (data_list.transform(tfms=tfms, size=size)
                .databunch(bs=batch_size)
                .normalize(imagenet_stats))
    return data

## **Eval**

In [ ]:
learn = create_cnn(get_data(512, 32), models.resnet50, metrics=[accuracy, mean_squared_error, mean_absolute_error], pretrained=True)
learn.model_dir = ROOT/'models'
learn.load('resnet_512a')

In [10]:
%%time
preds = learn.get_preds(ds_type=DatasetType.Test)

CPU times: user 56.1 s, sys: 39.3 s, total: 1min 35s
Wall time: 26min 42s


In [14]:
final_preds = []
for pred in (preds[0].tolist()):
    final_preds.append(round(pred[0],4))

In [15]:
output = pd.DataFrame({'filename':test_df.filename, 'xRot': final_preds})
output.to_csv('submission_512.csv', index=False)
output.head()

,filename,xRot
0,009999.png,214.6103
1,010000.png,199.4861
2,010001.png,329.8759
3,010002.png,128.8690
4,010003.png,151.6881


In [ ]:
from google.colab import files
files.download('submission_512.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>